In [1]:
import os, sys
sys.path.append('..')
import numpy as np
import pandas as pd
from glob import glob 
from skimage.io import imread, imsave
from easi_fish import n5_metadata_utils as n5mu
from easi_fish import roi_prop, spot, intensity
import importlib
importlib.reload(spot)
importlib.reload(intensity)

<module 'easi_fish.intensity' from 'D:\\SWAP\\code\\easifish-proc\\data_processing\\..\\easi_fish\\intensity.py'>

In [2]:
### Specify directories to data files
input_dir = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs"

# fixed image (directory - n5 format)
img_dir = os.path.join(input_dir, "r2\\export_sigma3.n5")
# segmentation mask (tif format accepted here)
lb_dir  = os.path.join(input_dir, "r2\\segmentation\\lt171_gene_4tile_r2-c3.tif" ) 
# bad roi
bad_roi = os.path.join(input_dir, "test-aug4\\bad_roi_list.npy")
# spot and output
spot_dir = os.path.join(input_dir, "r2\\spots")
### end input


# output
out_goodroi = os.path.join(spot_dir, "roi.csv") 
out_spots = os.path.join(spot_dir, "spots_r2.csv")
out_intensity = os.path.join(spot_dir, "intensity_c0_r2.csv")

In [3]:
### Load segmentation mask
### Get list of ROIs that are fully or partially outside the mask 
### Make sure to only include ROIs that are intact and in the overlapping regions across all rounds of FISH
lb=imread(lb_dir)
roi = np.max(lb)
print("number of ROIs in total:",roi)

bad_roi_list=np.load(bad_roi)
print('number of bad ROIs:', len(bad_roi_list))

number of ROIs in total: 13758
number of bad ROIs: 892


In [4]:
### Extract ROI properties
print(lb.shape)
lb = lb[500:600,500:600,500:600]
# lb.shape # z, x,y 

(1019, 915, 916)


In [5]:
%%time
### Extract ROI properties
df = roi_prop.roi_prop(lb)

C:\Users\zeiss\anaconda3\envs\easifish\lib\site-packages\skimage\measure\_regionprops.py:395: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6154 Qhull precision error: Initial simplex is flat (facet 1 is coplanar with the interior point)

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 829566882  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width 10  Error-roundoff 1e-14  _one-merge 7.2e-14  _near-inside 3.6e-13
  Visible-distance 2.1e-14  U-max-coplanar 2.1e-14  Width-outside 4.1e-14
  _wide-facet 1.2e-13  _maxoutside 8.2e-14

The input to qhull appears to be less than 3 dimensional, or a
computation has overflowed.

Qhull could not construct a clearly convex simplex from points:
- p1(v4):     0     1     0
- p8(v3):     3     2     0
- p25(v2):    10     0     0
- p0(v1):     0     0     0

The center point is coplanar with a facet, or a vertex is coplanar
with a neighboring facet. 

CPU times: total: 46.5 s
Wall time: 2.63 s


C:\Users\zeiss\anaconda3\envs\easifish\lib\site-packages\skimage\measure\_regionprops.py:395: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6154 Qhull precision error: Initial simplex is flat (facet 1 is coplanar with the interior point)

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 829566882  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width  4  Error-roundoff 4.3e-15  _one-merge 3e-14  _near-inside 1.5e-13
  Visible-distance 8.5e-15  U-max-coplanar 8.5e-15  Width-outside 1.7e-14
  _wide-facet 5.1e-14  _maxoutside 3.4e-14

The input to qhull appears to be less than 3 dimensional, or a
computation has overflowed.

Qhull could not construct a clearly convex simplex from points:
- p1(v4):     0     0     2
- p7(v3):     1     0     4
- p3(v2):     1     0     0
- p0(v1):     0     0     1

The center point is coplanar with a facet, or a vertex is coplanar
with a neighboring facet.  

In [6]:
### Remove bad ROIs and save the file
df_filtered = df.loc[~df['roi'].isin(bad_roi_list)]
df_filtered.roi = df_filtered.roi.astype(int)
df_filtered.to_csv(out_goodroi)

C:\Users\zeiss\AppData\Local\Temp\ipykernel_8128\4290796699.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.roi = df_filtered.roi.astype(int)


In [7]:
### Lipofuscin subtraction (optional)
# Load spot files for lipofuscin subtraction
channel_1=np.loadtxt(os.path.join(spot_dir, 'spots_c0.txt'), delimiter=',')
channel_2=np.loadtxt(os.path.join(spot_dir, 'spots_c1.txt'), delimiter=',')
# Identify lipofuscin spots
c0,c1,pAind,pBind=spot.rm_lipofuscin(channel_1,channel_2)
# Remove lipofuscin spots
true_pos_c0 = np.delete(channel_1, pAind, axis=0)
true_pos_c1 = np.delete(channel_2, pBind, axis=0)

In [8]:
### spot extraction and save file
spotcount=spot.spot_counts(lb, spot_dir, verbose=False)
spotcount.to_csv(out_spots)

Load: D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r2\spots\spots_c0.txt
Load: D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r2\spots\spots_c1.txt
Load: D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r2\spots\spots_c2.txt
Load: D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r2\spots\spots_c4.txt


In [9]:
### Intensity measurement and save file
df_intensity=intensity.measure_intensity(lb, img_dir, 'c0', cropdim=(500,600,500,600,500,600))
df_intensity.to_csv(out_intensity)